### Top 50 Spotify Tracks of 2020

**Importing and reading the data**

In [1]:
# import zipfile
# zip_dir = 'E:/archive.zip'
# extr_dir = 'Spottify_data'
# with zipfile.ZipFile(zip_dir, 'r') as zip_file:
#     zip_file.extractall(extr_dir)

In [28]:
import pandas as pd
import numpy as np

In [29]:
df = pd.read_csv(r'Spottify_data\spotifytoptracks.csv', index_col = 0)

**Ispecting the data**

<span style='color: blue;'>What is the size of the table? i.e. how many observations and features</span>

In [30]:
print(f'There are {df.shape[0]} observations and {df.shape[1]} features')

There are 50 observations and 16 features


<span style='color: blue;'>What are the names of the columns?</span>

In [31]:
df.columns

Index(['artist', 'album', 'track_name', 'track_id', 'energy', 'danceability',
       'key', 'loudness', 'acousticness', 'speechiness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'genre'],
      dtype='object')

<span style='color: blue;'>Get a glimpse of what it looks like</span>

In [32]:
df.head(3)

,artist,album,track_name,track_id,energy,danceability,key,loudness,acousticness,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,genre
0,The Weeknd,After Hours,Blinding Lights,0VjIjW4GlUZAMYd2vXMi3b,0.730,0.514,1,-5.934,0.00146,0.0598,0.000095,0.0897,0.334,171.005,200040,R&B/Soul
1,Tones And I,Dance Monkey,Dance Monkey,1rgnBhdG2JDFTbYkYRZAku,0.593,0.825,6,-6.401,0.68800,0.0988,0.000161,0.1700,0.540,98.078,209755,Alternative/Indie
2,Roddy Ricch,Please Excuse Me For Being Antisocial,The Box,0nbXyq5TXYPCO7pr3N8S4I,0.586,0.896,10,-6.687,0.10400,0.0559,0.000000,0.7900,0.642,116.971,196653,Hip-Hop/Rap


<span style='color: blue;'>What kind of data is stored in each column?</span>

In [33]:
df.dtypes

artist               object
album                object
track_name           object
track_id             object
energy              float64
danceability        float64
key                   int64
loudness            float64
acousticness        float64
speechiness         float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
duration_ms           int64
genre                object
dtype: object

<span style='color: blue;'>What are categorical features and how many unique entries are there in each of them?</span>

In [34]:
print("Categorical features:\n", df.select_dtypes(include='object').nunique())

Categorical features:
 artist        40
album         45
track_name    50
track_id      50
genre         16
dtype: int64


<span style='color: blue;'>What are numerical features?</span>

In [35]:
print("Numerical features:\n" + "\n".join(df.select_dtypes(include = 'number').columns.tolist()))

Numerical features:
energy
danceability
key
loudness
acousticness
speechiness
instrumentalness
liveness
valence
tempo
duration_ms


<span style='color: blue;'>Is there any missing data?</span>

In [36]:
if np.count_nonzero(df.isna()) == 0:
    print(" There is no missing data")
else:
    x = np.count_nonzero(df.isna())
    print(f'There are {x} missing data cases')

 There is no missing data


<span style='color: blue;'>Are there any duplicates? </span> <br>In this case, track name and/or track ID should be checked. 

In [37]:
if df.duplicated(subset=['track_name','track_id']).sum() == 0:
    print('There are no duplicated entries')
else:
    x = df.duplicated(subset=['track_name','track_id']).sum()
    print(f'There are {x} duplicated entries')

There are no duplicated entries


There are none but I already knew that from counting unique categorical feature entries

<span style='color: blue;'>Are there any outliers?</span> <br> My preffered way to check is boxplots. That works just fine if dataframe doesn't have too many features. If it was big, would probably go for interquartile range

In [38]:
import plotly.express as px

In [39]:
numerical_df = df.select_dtypes(include='number')

In [40]:
px.box(numerical_df, title="Box Plots for Numerical Columns")

This is not very clear. Need to make subplots

In [41]:
from plotly.subplots import make_subplots

In [42]:
fig = make_subplots(rows = len(numerical_df), cols = 1)

In [43]:
for i, col in enumerate(numerical_df, start=1):
    fig.add_trace(px.box(numerical_df, y=col).data[0], row=i, col=1)
    fig.update_yaxes(title_text=col, row=i, col=1)

In [44]:
fig.update_layout(
    title="Box Plots for Numerical Columns",
    showlegend=False,
    height=400 * len(numerical_df))

OK there's a bunch of outliers. Are they "real" though? Could be just a consequence of different genres, not mistakes. Honestly, in real life situation I would leave those untouched unless there was something crazy like duration 100 miliseconds

<span style='color: blue;'>Are there any artists that have more than 1 popular track? If yes, which and how many?</span>

In [27]:
pop_artist = df['artist'].value_counts()
print(f'These are the artists with more than one song in TOP50:\n{pop_artist[pop_artist > 1]}')

These are the artists with more than one song in TOP50:
Billie Eilish    3
Dua Lipa         3
Travis Scott     3
Justin Bieber    2
Harry Styles     2
Lewis Capaldi    2
Post Malone      2
Name: artist, dtype: int64


<span style='color: blue;'>Who was the most popular artist?</span>

In [21]:
print(f'These are the most popular Spotify Top 50 2020 artists:\n{pop_artist[pop_artist == pop_artist.max()]}') 

These are the most popular Spotify Top 50 2020 artists:
Billie Eilish    3
Dua Lipa         3
Travis Scott     3
Name: artist, dtype: int64


<span style='color: blue;'>How many artists in total have their songs in the top 50?</span>

In [22]:
print(f'{df["artist"].nunique()} artists have their songs in the TOP 50')

40 artists have their songs in the TOP 50


<span style='color: blue;'>Are there any albums that have more than 1 popular track? If yes, which and how many?</span>

In [23]:
pop_albums = df['album'].value_counts()
print(f'The albums with more than one song in TOP50:\n{pop_albums[pop_albums > 1]}')

The albums with more than one song in TOP50:
Future Nostalgia        3
Hollywood's Bleeding    2
Fine Line               2
Changes                 2
Name: album, dtype: int64


<span style='color: blue;'> How many albums in total have their songs in the top 50?</span>

In [24]:
print(f'There are {df["album"].nunique()} albums in TOP 50')

There are 45 albums in TOP 50


<span style='color: blue;'> Which tracks have a danceability score above 0.7?</span>

In [34]:
dance_mask = df['danceability'].gt(0.7)
print(f'These songs are realy good for dancing:\n {df[dance_mask][["artist","track_name"]]}')

These songs are realy good for dancing:
              artist                                     track_name
1       Tones And I                                   Dance Monkey
2       Roddy Ricch                                        The Box
3         SAINt JHN                          Roses - Imanbek Remix
4          Dua Lipa                                Don't Start Now
5            DaBaby                   ROCKSTAR (feat. Roddy Ricch)
7             Powfu               death bed (coffee for your head)
8     Trevor Daniel                                        Falling
10          KAROL G                                           Tusa
13        Lil Mosey                                Blueberry Faygo
14    Justin Bieber                       Intentions (feat. Quavo)
15            Drake                                   Toosie Slide
17         Doja Cat                                         Say So
18         Maroon 5                                       Memories
19           Future  

<span style='color: blue;'> Which tracks have a danceability score below 0.4?</span>

In [35]:
nodance_mask = df['danceability'].lt(0.4)
print(f'These songs are nice but not for dancing:\n{df[nodance_mask][["artist", "track_name"]]}')

These songs are nice but not for dancing:
           artist            track_name
44  Billie Eilish  lovely (with Khalid)


<span style='color: blue;'> Which tracks have their loudness above -5?</span>

In [36]:
loud_mask = df['loudness'].gt(-5)
print(f'These songs are loud:\n {df[loud_mask][["artist", "track_name"]]}')

These songs are loud:
            artist                                     track_name
4        Dua Lipa                                Don't Start Now
6    Harry Styles                               Watermelon Sugar
10        KAROL G                                           Tusa
12    Post Malone                                        Circles
16  Lewis Capaldi                                  Before You Go
17       Doja Cat                                         Say So
21   Harry Styles                                      Adore You
23       24kGoldn                         Mood (feat. iann dior)
31       Dua Lipa                                 Break My Heart
32            BTS                                       Dynamite
33          BENEE               Supalonely (feat. Gus Dapperton)
35      Lady Gaga                Rain On Me (with Ariana Grande)
37    Post Malone  Sunflower - Spider-Man: Into the Spider-Verse
38         Maluma                                          Hawái
39

<span style='color: blue;'> Which tracks have their loudness below -8?</span>

In [37]:
quiet_mask = df['loudness'].lt(-8)
print(f'These songs are quiet:\n {df[quiet_mask][["artist", "track_name"]]}')

These songs are quiet:
            artist                                      track_name
7           Powfu                death bed (coffee for your head)
8   Trevor Daniel                                         Falling
15          Drake                                    Toosie Slide
20      Jawsh 685                Savage Love (Laxed - Siren Beat)
24  Billie Eilish                             everything i wanted
26  Billie Eilish                                         bad guy
36   Travis Scott                             HIGHEST IN THE ROOM
44  Billie Eilish                            lovely (with Khalid)
47        JP Saxe  If the World Was Ending - feat. Julia Michaels


<span style='color: blue;'> Which track is the longest?<span>

In [42]:
longest = df['duration_ms'].idxmax()
print(f'This is the longest song in TOP 50:\n{df.loc[[longest], ["artist", "track_name"]]}')

This is the longest song in TOP 50:
          artist  track_name
49  Travis Scott  SICKO MODE


<span style='color: blue;'>Which track is the shortest?

In [43]:
shortest = df['duration_ms'].idxmin()
print(f'This is the shortest song in TOP 50:\n{df.loc[[shortest], ["artist", "track_name"]]}')

This is the shortest song in TOP 50:
      artist              track_name
23  24kGoldn  Mood (feat. iann dior)


<span style='color: blue;'>Which genre is the most popular?

In [44]:
print(f'{df["genre"].value_counts().idxmax()} is the most popular genre in TOP 50')

Pop is the most popular genre in TOP 50


<span style='color:blue;'>Which genres have just one song on the top 50?

In [45]:
genre_count = df["genre"].value_counts()
print('These genres have only 1 song in TOP 50:\n' +"\n".join(genre_count[genre_count == 1].index.tolist()))

These genres have only 1 song in TOP 50:
Nu-disco
R&B/Hip-Hop alternative
Pop/Soft Rock
Pop rap
Hip-Hop/Trap
Dance-pop/Disco
Disco-pop
Dreampop/Hip-Hop/R&B
Alternative/reggaeton/experimental
Chamber pop


<span style='color:blue;'>How many genres in total are represented in the top 50?

In [47]:
print(f'{df["genre"].nunique()} genres are represented in TOP 50')

16 genres are represented in TOP 50


<span style='color:blue;'>Which features are strongly positively correlated? <br> Basically that means finding not signifficant 

In [79]:
cor_matrix = round(numerical_df.corr(), 2)
mask = (cor_matrix > 0.5)&(cor_matrix < 1)
cor_matrix[mask].stack()

energy    loudness    0.79
loudness  energy      0.79
dtype: float64

The p value should be checked to make sure energy and loudness a truly correlated

In [80]:
import scipy.stats
scipy.stats.pearsonr(numerical_df['energy'], numerical_df['loudness'])

PearsonRResult(statistic=0.7916395653045618, pvalue=7.674982027015801e-12)

**There is a strong positive correlation between song loudness and energy (p < 0.001), meaning that the bigger loudness feature, the greater song energy**

<span style='color:blue;'>Which features are strongly negatively correlated?

In [83]:
mask_negative = cor_matrix.lt(-0.5)
cor_matrix[mask_negative].stack()

energy            acousticness       -0.68
loudness          instrumentalness   -0.55
acousticness      energy             -0.68
instrumentalness  loudness           -0.55
dtype: float64

In [87]:
print(scipy.stats.pearsonr(numerical_df['energy'], numerical_df['acousticness']))
print(scipy.stats.pearsonr(numerical_df['loudness'], numerical_df['instrumentalness']))

PearsonRResult(statistic=-0.682478520324153, pvalue=4.795622803271117e-08)
PearsonRResult(statistic=-0.5537348090851062, pvalue=3.0280913330404932e-05)


**There is a strong negative correlation (p < 0.001) between song energy and acousticness and between loudness and instrumentalness, meaning that loud, fast songs(i.e. high in energy) are low in acoustics, and loud song are low in instrumentalness**

<span style='color:blue;'>Which features are not correlated?</span> <br>
    Basically that means finding all not significant p values

In [128]:
from itertools import combinations
column_names = numerical_df.columns.tolist()
column_pairs = list(combinations(column_names, 2))
def p_notsignif(df = numerical_df, between = column_pairs):
    p_value_dict = {}
    for pair in between:
        col1, col2 = pair
        correlation, p_value = scipy.stats.pearsonr(df[col1], df[col2])
        if p_value >0.05:
            if col1 in p_value_dict:
                p_value_dict[col1].append(col2)
            else:
                p_value_dict[col1] = [col2]
            if col2 in p_value_dict:
                p_value_dict[col2].append(col1)
            else:
                p_value_dict[col2] = [col1]
    return p_value_dict

In [129]:
p_notsignif()

{'energy': ['danceability',
  'key',
  'speechiness',
  'liveness',
  'tempo',
  'duration_ms'],
 'danceability': ['energy',
  'loudness',
  'speechiness',
  'instrumentalness',
  'liveness',
  'tempo',
  'duration_ms'],
 'key': ['energy',
  'loudness',
  'acousticness',
  'speechiness',
  'instrumentalness',
  'liveness',
  'valence',
  'tempo',
  'duration_ms'],
 'speechiness': ['energy',
  'danceability',
  'key',
  'loudness',
  'acousticness',
  'instrumentalness',
  'liveness',
  'valence',
  'tempo'],
 'liveness': ['energy',
  'danceability',
  'key',
  'loudness',
  'acousticness',
  'speechiness',
  'instrumentalness',
  'valence',
  'tempo',
  'duration_ms'],
 'tempo': ['energy',
  'danceability',
  'key',
  'loudness',
  'acousticness',
  'speechiness',
  'instrumentalness',
  'liveness',
  'valence',
  'duration_ms'],
 'duration_ms': ['energy',
  'danceability',
  'key',
  'loudness',
  'acousticness',
  'instrumentalness',
  'liveness',
  'valence',
  'tempo'],
 'loudness'

**In this dictionary we can see which features do not correlate**

<span style='color:blue;'>How does the danceability score compare between Pop, Hip-Hop/Rap, Dance/Electronic, and Alternative/Indie genres?

In [151]:
genre_filter = ['Pop', 'Hip-Hop/Rap', 'Dance/Electronic', 'Alternative/Indie']
compare_dance = df[df['genre'].isin(genre_filter)].filter(items = ['danceability', 'genre'])
compare_dance.groupby('genre')['danceability'].mean().round(3).sort_values(ascending=False)

genre
Hip-Hop/Rap          0.766
Dance/Electronic     0.755
Pop                  0.678
Alternative/Indie    0.662
Name: danceability, dtype: float64

**Hip-Hop/Rap has the highest mean danceability, closely followed by Dance/Electronics** It might be nicer to see the differences between these genres in a figure

In [150]:
px.box(compare_dance,x = 'genre', y = 'danceability', title="Comparison of danceability")

**We can see that Dance/Electronics has higher danceability median and lower variation than Hip-Hop/Rap, making this genre more 'dancy' than the rest. Both mean and median danceability of Alternative/Indie is the lowest, however based on the boxplot we can see that there's 25% chance to come across realy 'dancy' Alternative/Indie song. Pop is scored slightly higher in danceability than Alternative/Indie overall**

<span style='color:blue;'>How does the loudness score compare between Pop, Hip-Hop/Rap, Dance/Electronic, and Alternative/Indie genres?

In [160]:
compare_loud = df[df['genre'].isin(genre_filter)].filter(items = ['loudness', 'genre'])
compare_loud.groupby('genre')['loudness'].mean().round(2).sort_values(ascending=False)
# compare_loud

genre
Dance/Electronic    -5.34
Alternative/Indie   -5.42
Pop                 -6.46
Hip-Hop/Rap         -6.92
Name: loudness, dtype: float64

In [161]:
px.box(compare_loud,x = 'genre', y = 'loudness', title="Comparison of loudness")

In [164]:
df.loc[df['loudness'] == min(compare_loud['loudness'])]

,artist,album,track_name,track_id,energy,danceability,key,loudness,acousticness,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,genre
24,Billie Eilish,everything i wanted,everything i wanted,3ZCTVFBt2Brf31RLEnCkWJ,0.225,0.704,6,-14.454,0.902,0.0994,0.657,0.106,0.243,120.006,245426,Pop


**Hip-Hop/Rap has the lowest mean and median loudness score, and Dance/Electronics is the loudest. One song, "Everything I wanted" By Billie Eilish stands out as particulary quiet. After having a closer look at it we can see that it is also highly instrumental, and we saw from correlation analysis that loudness and instrumentalness are highly negatively correlated. Therefore the low loudness score is most likely not some mistake but a true feature of this song**

<span style='color:blue;'>How does the acousticness score compare between Pop, Hip-Hop/Rap, Dance/Electronic, and Alternative/Indie genres?

In [166]:
compare_acoustic = df[df['genre'].isin(genre_filter)].filter(items = ['acousticness', 'genre'])
compare_acoustic.groupby('genre')['acousticness'].mean().round(3).sort_values(ascending=False)

genre
Alternative/Indie    0.584
Pop                  0.324
Hip-Hop/Rap          0.189
Dance/Electronic     0.099
Name: acousticness, dtype: float64

In [168]:
px.box(compare_acoustic,x = 'genre', y = 'acousticness', title="Comparison of acousticness")

**Alternative/Indie stands out as highest in acousticness. As should be expected, Dance/Electronics is the least acoustic. Hip-Hop/Rap in general is not very acoustic genre, with one exception. Pop group has 3 outliers - song with high acousticness score, which makes group mean higher than median**

In [51]:
type(cor_matrix)

pandas.core.frame.DataFrame

OK there's a bunch of outliers. Are they "real" though? Could be just a consequence of different genres, not mistakes. Honestly, in real life situation I would leave those untouched unless there was something crazy like duration 100 miliseconds

In [20]:
np.count_nonzero(df['genre'] == 'Hip-Hop/Rap')#tik patikrinti kaip teisingai pasirinkti

13

In [170]:
print(f"There are {np.count_nonzero(df['genre'] == 'Hip-Hop/Rap')} Hip-Hop/Rap songs in TOP50")

There are 13 Hip-Hop/Rap songs in TOP50


In [19]:
type(pop_artist)

pandas.core.series.Series

In [20]:
pop_artist.index

Index(['Billie Eilish', 'Dua Lipa', 'Travis Scott', 'Justin Bieber',
       'Harry Styles', 'Lewis Capaldi', 'Post Malone', 'Cardi B', 'Surfaces',
       'Eminem', 'BTS', 'BENEE', 'Surf Mesa', 'Lady Gaga', 'The Weeknd',
       'Shawn Mendes', 'Regard', 'Black Eyed Peas', 'THE SCOTTS', 'Bad Bunny',
       'Juice WRLD', 'Ariana Grande', 'Maluma', 'Topic', '24kGoldn',
       'Tones And I', 'Jawsh 685', 'Future', 'Maroon 5', 'Doja Cat', 'Drake',
       'Lil Mosey', 'Arizona Zervas', 'KAROL G', 'Trevor Daniel', 'Powfu',
       'DaBaby', 'SAINt JHN', 'Roddy Ricch', 'JP Saxe'],
      dtype='object')

In [98]:
df

,artist,album,track_name,track_id,energy,danceability,key,loudness,acousticness,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,genre
0,The Weeknd,After Hours,Blinding Lights,0VjIjW4GlUZAMYd2vXMi3b,0.730,0.514,1,-5.934,0.00146,0.0598,0.000095,0.0897,0.3340,171.005,200040,R&B/Soul
1,Tones And I,Dance Monkey,Dance Monkey,1rgnBhdG2JDFTbYkYRZAku,0.593,0.825,6,-6.401,0.68800,0.0988,0.000161,0.1700,0.5400,98.078,209755,Alternative/Indie
2,Roddy Ricch,Please Excuse Me For Being Antisocial,The Box,0nbXyq5TXYPCO7pr3N8S4I,0.586,0.896,10,-6.687,0.10400,0.0559,0.000000,0.7900,0.6420,116.971,196653,Hip-Hop/Rap
3,SAINt JHN,Roses (Imanbek Remix),Roses - Imanbek Remix,2Wo6QQD1KMDWeFkkjLqwx5,0.721,0.785,8,-5.457,0.01490,0.0506,0.004320,0.2850,0.8940,121.962,176219,Dance/Electronic
4,Dua Lipa,Future Nostalgia,Don't Start Now,3PfIrDoz19wz7qK7tYeu62,0.793,0.793,11,-4.521,0.01230,0.0830,0.000000,0.0951,0.6790,123.950,183290,Nu-disco
5,DaBaby,BLAME IT ON BABY,ROCKSTAR (feat. Roddy Ricch),7ytR5pFWmSjzHJIeQkgog4,0.690,0.746,11,-7.956,0.24700,0.1640,0.000000,0.1010,0.4970,89.977,181733,Hip-Hop/Rap
6,Harry Styles,Fine Line,Watermelon Sugar,6UelLqGlWMcVH1E5c4H7lY,0.816,0.548,0,-4.209,0.12200,0.0465,0.000000,0.3350,0.5570,95.390,174000,Pop
7,Powfu,death bed (coffee for your head),death bed (coffee for your head),7eJMfftS33KTjuF7lTsMCx,0.431,0.726,8,-8.765,0.73100,0.1350,0.000000,0.6960,0.3480,144.026,173333,Hip-Hop/Rap
8,Trevor Daniel,Nicotine,Falling,2rRJrJEo19S2J82BDsQ3F7,0.430,0.784,10,-8.756,0.12300,0.0364,0.000000,0.0887,0.2360,127.087,159382,R&B/Hip-Hop alternative
9,Lewis Capaldi,Divinely Uninspired To A Hellish Extent,Someone You Loved,7qEHsqek33rTcFNT9PFqLf,0.405,0.501,1,-5.679,0.75100,0.0319,0.000000,0.1050,0.4460,109.891,182161,Alternative/Indie


kaip patikrinti kiek laiko trunka skaiciavimas

In [160]:
numbers = pd.Series(np.random.randint(0, 1000, 10000))

In [161]:
%%timeit -n 100
total = 0
for number in numbers:
    total+=number
total/len(numbers)

711 µs ± 12.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [162]:
%%timeit -n 100
total = np.sum(numbers)
total/len(numbers)

42.1 µs ± 16.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [177]:
!pip install c:/mano/nb_black-master/nb_black-master

Processing c:\mano\nb_black-master\nb_black-master
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for nb-black: filename=nb_black-1.0.7-py3-none-any.whl size=5367 sha256=a98a2312aaa202c89a1a6d4755ad4804913a9fb9ef672f9a2f35229d084c2b1a
  Stored in directory: c:\users\grazina s\appdata\local\pip\cache\wheels\e3\4a\69\5ee8802fd67a6793bca94284d260293b03c6220b9375c3cea5
Successfully built nb-black
  Attempting uninstall: nb-black
    Found existing installation: nb-black 1.0.7
    Uninstalling nb-black-1.0.7:
      Successfully uninstalled nb-black-1.0.7


In [179]:
!pip show nb_black

Name: nb-black
Version: 1.0.7
Summary: A simple extension for Jupyter Notebook and Jupyter Lab to beautify Python code automatically using Black.
Home-page: https://github.com/dnanhkhoa/nb_black
Author: Khoa Duong
Author-email: dnanhkhoa@live.com
License: MIT
Location: C:\mano\ANACONDA\envs\learn\Lib\site-packages
Requires: black, ipython
Required-by: 


In [1]:
!jupyter nbextension enable nb_black --user --py

Traceback (most recent call last):
  File "C:\mano\ANACONDA\Scripts\jupyter-nbextension-script.py", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "C:\mano\ANACONDA\Lib\site-packages\jupyter_core\application.py", line 277, in launch_instance
    return super().launch_instance(argv=argv, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\mano\ANACONDA\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\mano\ANACONDA\Lib\site-packages\notebook\nbextensions.py", line 972, in start
    super().start()
  File "C:\mano\ANACONDA\Lib\site-packages\jupyter_core\application.py", line 266, in start
    self.subapp.start()
  File "C:\mano\ANACONDA\Lib\site-packages\notebook\nbextensions.py", line 882, in start
    self.toggle_nbextension_python(self.extra_args[0])
  File "C:\mano\ANACONDA\Lib\site-packages\notebook\nbextensions.py", line 855, in toggle_nbextension_python
    return toggle(module

In [2]:
!pip list

Package           Version
----------------- -------
asttokens         2.2.1
backcall          0.2.0
black             23.7.0
click             8.0.4
colorama          0.4.6
decorator         5.1.1
executing         1.2.0
ipython           8.14.0
jedi              0.19.0
matplotlib-inline 0.1.6
mypy-extensions   0.4.3
nb-black          1.0.7
packaging         23.0
parso             0.8.3
pathspec          0.10.3
pickleshare       0.7.5
pip               23.2.1
platformdirs      2.5.2
prompt-toolkit    3.0.39
pure-eval         0.2.2
Pygments          2.15.1
setuptools        68.0.0
six               1.16.0
stack-data        0.6.2
traitlets         5.9.0
wcwidth           0.2.6
wheel             0.38.4
